# Библио

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from tqdm import tqdm
import torchvision
from IPython.display import clear_output

try:
    import transformers
except ModuleNotFoundError:
    !pip install transformers
    !pip install transformers sentencepiece --quiet
    clear_output()
    import transformers
from transformers import (
    T5ForConditionalGeneration, T5TokenizerFast, T5Tokenizer,
    DataCollatorForLanguageModeling, TrainingArguments, Trainer,
)

try:
    import datasets
except ModuleNotFoundError:
    !pip install datasets
    clear_output()
from datasets import load_dataset

try:
    import ignite.metrics
except ModuleNotFoundError:
    !pip install pytorch-ignite
    clear_output()
from ignite.metrics import Rouge

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Тренировочный текст (ВКР)

In [ ]:
f = open('/content/drive/MyDrive/Colab Notebooks/text.txt', 'r')
txt = f.read()

In [ ]:
print(txt)

Методы аспектного анализа научно-технических текстов


Введение

В настоящее время нам приходится ежедневно обрабатывать, просматривать и изучать огромное количество текстовой информации. Как следствие, актуальной является задача автоматического извлечения различной информации из текста (его смысла, жанра, стиля, содержания, мнения автора и т.д.) и ее дальнейшая обработка [4, 31], которая применяется во многих прикладных задачах: литературно-научное редактирование, реферирование и аннотирование, информационный поиск, определение тональности, жанровая классификация, машинный перевод [3, 6, 13].
Существуют различные виды анализа текстов, такие как извлечение именованных сущностей [32], анализ тональностей [1], дискурсивный анализ [15], включающий проверку связности текста [12], и другие. В рамках данной работы мы будем исследовать аспектный анализ.
Целью аспектного анализа является рассмотрение текста под определенным углом, например, с точки зрения содержания: все ли части (вступление, 

# ruT5 base absum

https://huggingface.co/cointegrated/rut5-base-absum

In [ ]:
model_name = "cointegrated/rut5-base-absum"
tokenizer_1 = T5TokenizerFast.from_pretrained(model_name)
model_t5_1 = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
model_t5_1.cuda();
model_t5_1.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0,
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer_1(text, return_tensors='pt', padding=True).to(model_t5_1.device)
    with torch.inference_mode():
        out = model_t5_1.generate(
            **x,
            max_length=max_length, num_beams=num_beams,
            do_sample=do_sample, repetition_penalty=repetition_penalty,
            **kwargs
        )
    return tokenizer_1.decode(out[0], skip_special_tokens=True)

## Пример использования

In [ ]:
text = 'Высота башни составляет 324 метра (1063 фута), примерно такая же высота, как у 81-этажного здания, и самое высокое сооружение в Париже. Его основание квадратно, размером 125 метров (410 футов) с любой стороны. Во время строительства Эйфелева башня превзошла монумент Вашингтона, став самым высоким искусственным сооружением в мире, и этот титул она удерживала в течение 41 года до завершения строительство здания Крайслер в Нью-Йорке в 1930 году. Это первое сооружение которое достигло высоты 300 метров. Из-за добавления вещательной антенны на вершине башни в 1957 году она сейчас выше здания Крайслер на 5,2 метра (17 футов). За исключением передатчиков, Эйфелева башня является второй самой высокой отдельно стоящей структурой во Франции после виадука Мийо.'
print(len(tokenizer(text, return_tensors='pt', padding=True).to(model_t5_1.device)[0]))

233


In [ ]:
print(summarize(text))

Эйфелева башня достигла высоты 300 метров.


In [ ]:
print(summarize(text, n_words=100))

Французская Эйфелева башня достигла высоты 300 метров.


In [ ]:
print(summarize(text, compression=0.01))

Французская Эйфелева башня достигла высоты 300 метров.


## Пример на моей дипломной работе (10к+ токенов)

Длина в токенах:

In [ ]:
'''input_ids = tokenizer(
    [txt],
    add_special_tokens=True,
    return_tensors="pt"
)["input_ids"]
print(input_ids.size())'''

torch.Size([1, 10058])


Эксперименты:

In [ ]:
print(summarize(txt, n_words=100))

Основы подхода к извлечению информации из текстов с помощью аспектных индикаторов были заложены в 70-е годы


In [ ]:
print(summarize(txt))

Основы подхода к извлечению информации из текстов с помощью аспектных индикаторов были заложены в 70-е годы


In [ ]:
print(summarize(txt, max_length=11000, n_words=100))

Основы подхода к извлечению информации из текстов с помощью аспектных индикаторов были заложены в 70-е годы


In [ ]:
print(summarize(txt, max_length=11000, n_words=500))

Основы подхода к извлечению информации из текстов с помощью аспектных индикаторов были заложены в 70-е годы


***Промежуточные выводы***: смысл был утерян, генерирует весьма короткие рефераты. Это вытекает из длины эталонных рефератов при дообучении на датасет.

## Тесты на более коротких текстах (тезисы к ВКР)

Реферируем текст целиком:

In [ ]:
for i in range(64):
    t = open(f'/content/drive/MyDrive/Colab Notebooks/texts/text_{i}', 'r', encoding='cp1251').read()
    print(f'{i}) {summarize(t)}')
    print('-----------')

0) Выпускная квалификационная работа посвящена численному исследованию математических моделей процесса деионизации водного раствора.
-----------
1) Построение алгоритмов модификации ТМ произвольной размерности
-----------
2) Разработка адаптивного метода сегментации гистологических изображений желез
-----------
3) Методы подавления шума на изображениях используют метод Перона—Малика, основанный на использовании анизотропной диффузии.
-----------
4) Метод автоматического анализа минерального состава руд по фотографиям аншлифов
-----------
5) Метод трекинга актиновых филаментов на микроскопических изображениях
-----------
6) Метод сегментации изображений радужной оболочки глаза
-----------
7) Сравнение методов граничных интегральных уравнений с слабой и сильной особенностями в задачах электромагнитного рассеяния
-----------
8) В настоящей работе были разработаны методы определения наличия или отсутствия патологии кровотока в кровеносной системе.
-----------
9) В настоящей работе проводил

Через реферирование абзацев:

In [ ]:
for i in range(64):
    txts = open(f'/content/drive/MyDrive/Colab Notebooks/texts/text_{i}', 'r', encoding='cp1251').read().split('\n')
    print(f'({i})')
    for t in txts:
        if t:
          print(summarize(t))
    print('-----------')

(0)
В работе рассматривается метод опреснения воды в сравнении с методами обратного осмоса.
Метод емкостной деионизации заключается в прохождении раствора через электрохимическую ячейку между двумя пористыми электродами.
В работе предложены условия равновесия, переноса и диффузии водных растворов.
Выпускная квалификационная работа посвящена численному исследованию математических моделей емкостной деионизации.
В работе были рассмотрены две математические модели процесса деионизации водного раствора, представляющие собой начально-краевые задачи для системы уравнений в частных производных.
-----------
(1)
Улучшение численных гарантированных оценок множеств решений обыкновенных дифференциальных уравнений.
В области динамических систем используются доказательные вычисления (ДВ) — вычисления, направленные на доказательство новых фактов и теорем.
Для доказательства существования периодических траекторий в системах ОДУ могут быть использованы методы, основанные на интервальных моделях Тейлора.

# ruT5 base sum_gazeta

https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta

In [ ]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

tokenizer_2 = T5TokenizerFast.from_pretrained(model_name)
model_t5_2 = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

## Пример на дипломной работе

In [ ]:
def summarize_2(text):

    input_ids = tokenizer_2(
        [text],
        max_length=2048,
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )["input_ids"].to(model_t5_2.device)

    output_ids = model_t5_2.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4,
        max_length=2048,
    )[0]

    return tokenizer_2.decode(output_ids, skip_special_tokens=True)

Тест на текст ВКР:

In [ ]:
print(summarize_2(txt))

Методы аспектного анализа научно-технических текстов на русском языке были разработаны в рамках выпускной квалификационной работы, посвященной разработке методов анализа текста.


## Тесты на более коротких текстах (тезисы к ВКР)

Текст целиком:

In [ ]:
for i in range(64):
    t = open(f'/content/drive/MyDrive/Colab Notebooks/texts/text_{i}', 'r', encoding='cp1251').read()
    print(f'{i}) {summarize_2(t)}')
    print('-----------')

0) Выпускная квалификационная работа посвящена численному исследованию математических моделей емкостной деионизации и численному решению некоторых обратных задач.
-----------
1) В работе, посвященной доказательству существования и построению решений систем обыкновенных дифференциальных уравнений, показано, что множество решений можно описать даже за один период.
-----------
2) Адаптивный метод сегментации гистологических изображений показал наилучшие результаты среди нескольких опробованных в ходе разработки алгоритмов, позволяющих в короткие сроки получить точные результаты для большого объема данных. В дальнейшем планируется тестировать алгоритм на наборе реальных данных PATH-DT-MSU.
-----------
3) Нелинейные методы подавления шума на изображениях показали хорошие результаты по автоматическому поиску оптимальных параметров алгоритма взвешенной ядерной нормы. Это позволяет убрать шум, потеряв при этом минимум информации.
-----------
4) Метод сегментации изображений аншлифов может быть

С реферированием по абзацам:

In [ ]:
for i in range(64):
    txts = open(f'/content/drive/MyDrive/Colab Notebooks/texts/text_{i}', 'r', encoding='cp1251').read().split('\n')
    print(f'({i})')
    for t in txts:
        if t:
          print(summarize_2(t))
    print('-----------')

(0)
Метод емкостной деионизации — менее энергоемкий и экономичный метод опреснения воды в сравнении с методами обратного осмоса и дистилляции.
Метод емкостной деионизации рассмотрен в работе, посвященной методу деионизации водного раствора через электрохимическую ячейку между двумя пористыми электродами. В работе представлены аэрогелные электроды, как поверхности конденсатора большой емкости собирающего ионы из промежутка между электродами, между которыми задается определённая разность потенциалов.
В работе, посвященной деионизации водных растворов, предложены уравнения, начальные и краевые условия, описывающие деионизацию растворов. В работе предложены условия равновесия, переноса и диффузии ионов, алгоритм количественного расчета условий равновесия и переноса ионов.
Выпускная квалификационная работа посвящена численному исследованию математических моделей емкостной деионизации и численному решению некоторых обратных задач.
В работе была рассмотрена модель процесса деионизации водного

**Выводы**: вторая модель была дообучена только на 1 датасете - GAZETA, в которой эталонные рефераты длиннее, чем в других датасетах (русскоязычных частях мультиязычных корпусов, на которых также дообучалась первая модель), а потому рефераты получаются более длинными и все предложения закончены. Кроме того, создается впечатление, что данная языковая модель генерирует более качественные рефераты с точки зрения смысла.

# Дообучение на задачу реферирования

## Встроенный тренер с hf (для модели ruT5 base sum_gazeta):

Посмотреть размеры существующих в коллекции текстов:

In [ ]:
train_set = []

for i in range(1, 11):
    with open(f'{path}/dataset/{i}.txt') as f:
        data = f.read()
    data = data.split('\n\n')
    text, ref = data[1], data[2]
    print(tokenizer_2([text],add_special_tokens=True,return_tensors="pt")["input_ids"].size())
    train_set.append((text, ref))

torch.Size([1, 2370])
torch.Size([1, 4981])
torch.Size([1, 2519])
torch.Size([1, 5775])
torch.Size([1, 2459])
torch.Size([1, 2205])
torch.Size([1, 2703])
torch.Size([1, 3768])
torch.Size([1, 2863])
torch.Size([1, 2446])


Функция для создания датасета (добавить в эталонный реферат в метки (эталонные рефераты)):

In [ ]:
def make_dataset(data):
    txt, sum = data
    txt = tokenizer_2(txt, max_length=5800, padding="max_length", truncation=True)
    sum = tokenizer_2(sum, max_length=5800, padding="max_length", truncation=True).input_ids
    sum = [label if label != 0 else -100 for label in sum]
    txt["labels"] = sum
    return txt

In [ ]:
train_dataset = list(map(make_dataset, train_set))

Обучение:

In [ ]:
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer_2, mlm=False)

In [ ]:
# параметры для Тренера
training_args = TrainingArguments(
    output_dir="t5-model",
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    warmup_steps=10,
    gradient_accumulation_steps=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Тренер
trainer = Trainer(
    model=model_t5_2,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer_2,
    data_collator=data_collator,
    optimizers = (torch.optim.AdamW(model_t5_2.parameters(),lr=1e-5), None)
)

In [ ]:
# запускаем обучение
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: ignored

Для base моделей для обучения не хватает даже pro версии (ни встроенными, ни ручными методами - последние тоже смотрелись, но соответствующие ячейки кода перешли в дообучение модели поменьше) - перешли на small версию.

## ruT5 small

### Модель без дообучения:

https://huggingface.co/cointegrated/rut5-small

In [ ]:
model_name = "cointegrated/rut5-small"

tokenizer_3 = T5Tokenizer.from_pretrained(model_name)
model_t5_3 = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


### Загрузка модели с диска:

In [ ]:
saved_model_path = path + '/t5-small'
model_name = "cointegrated/rut5-small"

tokenizer_4 = T5Tokenizer.from_pretrained(model_name)
model_t5_4 = T5ForConditionalGeneration.from_pretrained(saved_model_path).to(device)

## Дообучение на датасете GAZETA на задачу реферирования:

https://huggingface.co/datasets/IlyaGusev/gazeta

In [ ]:
dataset_train = load_dataset('IlyaGusev/gazeta', revision="v2.0")["train"]
dataset_val = load_dataset('IlyaGusev/gazeta', revision="v2.0")["validation"]
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v2.0")["test"]

clear_output()

Длины рефератов в токенах для train-val-test выборок:

In [ ]:
def count_dataset_(data):
    max_len = 0
    for inst in data:
        l = len(tokenizer_3(inst['summary'], add_special_tokens=True).input_ids)
        max_len = l if l > max_len else max_len
    return max_len

print(count_dataset_(dataset_train))
print(count_dataset_(dataset_val))
print(count_dataset_(dataset_test))

174
168
170


Довольно жадный, но простой способ, что съест 10 ГБ памяти:

In [ ]:
'''def make_dataset_(data):
    dataset = []
    max_len = 0
    for inst in data:
        l = len(tokenizer_3(inst['text'], add_special_tokens=True).input_ids)
        max_len = l if l > max_len else max_len
        txt = tokenizer_3(inst['text'], add_special_tokens=True, max_length=1500, padding="max_length", truncation=True)
        sum = tokenizer_3(inst['summary'], add_special_tokens=True, max_length=1500, padding="max_length", truncation=True).input_ids
        txt["labels"] = sum
        dataset.append(txt)
    return dataset, max_len'''

def make_dataset_(data):
    dataset = []
    for inst in data:
        txt = tokenizer_3(inst['text'], add_special_tokens=True, max_length=2890, padding="max_length", truncation=True)
        sum = tokenizer_3(inst['summary'], add_special_tokens=True, max_length=200, padding="max_length", truncation=True).input_ids
        txt["labels"] = sum
        dataset.append(txt)
    return dataset

In [ ]:
'''
dataset_train, max_tr = make_dataset_(dataset_train)
dataset_val, max_v = make_dataset_(dataset_val)
dataset_test, max_te = make_dataset_(dataset_test)
'''
dataset_train = make_dataset_(dataset_train)
dataset_val = make_dataset_(dataset_val)

set : num of instances : max len in tokens

In [ ]:
'''
print(f'train: {len(dataset_train)} : {max_tr}')
print(f'  val: {len(dataset_val)} : {max_v}')
print(f' test: {len(dataset_test)} : {max_te}')
'''

train: 60964 : 2886
  val: 6369 : 2827
 test: 6793 : 2818


### 1/2 эпоха обучения

!! Код ниже предназначен для запуска обучения по частям. !!

In [ ]:
saved_model_path = path

Загрузка последней версии модели и оптимизатора:

In [ ]:
model_name = "cointegrated/rut5-small"

tokenizer_3 = T5Tokenizer.from_pretrained(model_name)
model_t5_3 = T5ForConditionalGeneration.from_pretrained(model_name)
optimizer = torch.optim.AdamW(model_t5_3.parameters(),lr=1e-4)

checkpoint = torch.load(saved_model_path + '/try/model_t5_small.pth', map_location='cpu')
model_t5_3.load_state_dict(checkpoint['model_state_dict'])
model_t5_3.to(device)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Сохранение:

([Обсуждение](https://github.com/huggingface/transformers/issues/7849) в github transformers, почему модель работает после сохранения хуже (потому что не сохраняется оптимизатор))

In [ ]:
def save(model, optimizer):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, saved_model_path + '/try/model_t5_small.pth')

Одна промежуточная часть:

In [ ]:
part_1 = load_dataset('IlyaGusev/gazeta', split='train[50%:75%]', revision="v2.0")
part_1 = make_dataset_(part_1)

!! Код следующей ячейки стоит запускать при возникновении ошибке в ячейке ниже данной. После этого будет необходимо **перезапустить** ноутбук целиком

In [ ]:
# if "Using the `Trainer` with `PyTorch` requires `accelerate`: Run `pip install --upgrade accelerate`" occured

! pip uninstall -y transformers accelerate
! pip install transformers accelerate
clear_output()
# then reload notebook

Обучение части датасета (без валидации):

In [ ]:
training_args = TrainingArguments(
    output_dir=saved_model_path+"/t5-model-small",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=1,
    warmup_steps=10,
    gradient_accumulation_steps=16,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model_t5_3,
    args=training_args,
    train_dataset=part_1,
    tokenizer=tokenizer_3,
    optimizers = (optimizer, None)
)

In [ ]:
model_t5_3.train()
trainer.train()

Step,Training Loss


TrainOutput(global_step=476, training_loss=1.1081771209460347, metrics={'train_runtime': 7057.6696, 'train_samples_per_second': 2.159, 'train_steps_per_second': 0.067, 'total_flos': 1.435584342786048e+16, 'train_loss': 1.1081771209460347, 'epoch': 1.0})

Созранение:

In [ ]:
save(model_t5_3, optimizer)

Последняя часть (с валидацией):

In [ ]:
part_1 = load_dataset('IlyaGusev/gazeta', split='train[-25%:]', revision="v2.0")
part_1 = make_dataset_(part_1)
dataset_val = load_dataset('IlyaGusev/gazeta', revision="v2.0")["validation"]
dataset_val = make_dataset_(dataset_val)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
training_args = TrainingArguments(
    output_dir=saved_model_path+"/t5-model-small",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    warmup_steps=10,
    gradient_accumulation_steps=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model_t5_3,
    args=training_args,
    train_dataset=part_1,
    eval_dataset=dataset_val,
    tokenizer=tokenizer_3,
    optimizers = (optimizer, None)
)

In [ ]:
model_t5_3.train()
trainer.train()

In [ ]:
save(model_t5_3, optimizer)

### 1 эпоха обучения
Обучение (batch_size=2 достаточно для обучения на ускарителе 12ГБ, но обучается примерно 27 часов на 4-х эпохах):

In [ ]:
model_t5_3.train()

In [ ]:
# параметры для Тренера
training_args = TrainingArguments(
    output_dir="t5-model-small",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    warmup_steps=10,
    gradient_accumulation_steps=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Тренер
trainer = Trainer(
    model=model_t5_3,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer_3,
    optimizers = (torch.optim.AdamW(model_t5_3.parameters(),lr=1e-4), None)
)

In [ ]:
# 1 эпоха
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,6.217100,1.508305


TrainOutput(global_step=952, training_loss=4.387304995240283, metrics={'train_runtime': 7475.5301, 'train_samples_per_second': 8.155, 'train_steps_per_second': 0.127, 'total_flos': 5.742337371144192e+16, 'train_loss': 4.387304995240283, 'epoch': 1.0})

In [ ]:
saved_model_path = '/content/drive/MyDrive/my_t5_small'
model_t5_3.save_pretrained(saved_model_path)

### Тесты модели по эпохам

Текущее состояние обученной модели:

In [ ]:
model_t5_3.train()
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,0.979064


TrainOutput(global_step=476, training_loss=1.093704608308167, metrics={'train_runtime': 8129.9696, 'train_samples_per_second': 1.875, 'train_steps_per_second': 0.059, 'total_flos': 1.435584342786048e+16, 'train_loss': 1.093704608308167, 'epoch': 1.0})

Тестирование на 1 тексте:

In [ ]:
model_t5_3.eval()

txt = '''В работе рассматривается метод емкостной деионизации — менее энергоемкий и экономичный метод опреснения воды в сравнении с методами обратного осмоса и дистилляции.
Суть метода емкостной деионизации подробно рассмотрена в работе. Он заключается в пропускании водного раствора через электрохимическую ячейку между двумя пористыми электродами, в качестве которых обычно используются углеродные электроды с высокой удельной поверхностью, между которыми задается определённая разность потенциалов. Теоретический подход сводится к представлению аэрогелных электродов, как поверхности конденсатора большой емкости  собирающего ионы из промежутка между электродами, по которому происходит движение раствора.
В работе  предложены уравнения, начальные и краевые условия, описывающие деионизацию водных растворов, также построен алгоритм количественного расчета условий равновесия, переноса и диффузии ионов.
Выпускная квалификационная работа посвящена численному исследованию математических моделей емкостной деионизации и численному решению, в рамках этих моделей, некоторых обратных задач.
В работе рассмотрены две математические модели процесса деионизации водного раствора, представляющие собой начально-краевые задачи для системы уравнений в частных производных. Модели различаются разным описанием процесса поглощения ионов. Так была рассмотрена упрощенная модель поглощения и модель более приближенная к реальной работе установки. Для каждой модели предложен и программно реализован численный метод, позволяющий вычислять концентрацию ионов в канале и порах. Численно исследована зависимость выходной концентрации ионов от различных параметров моделей. Рассмотрены обратные задачи состоящие в нахождении неизвестных кинетических коэффициентов, определяющих процесс поглощения ионов. Предложен и численно реализован метод поиска кинетических коэффициентов по интегральной выходной концентрации ионов.
'''

Предобученная чистая модель:

In [ ]:
input = tokenizer_3(txt, add_special_tokens=True, max_length=2890, padding="max_length", truncation=True, return_tensors='pt').to(device)
tokenizer_3.decode(model_t5_3.generate(input_ids=input.input_ids, attention_mask=input.attention_mask, max_length=500)[0], skip_special_tokens=True)

'Выпускная квалификационная работа рассматривается метод емкостной деионизации, которая именно построен алгоритм количественного расчета условий равновесия, переноса и диффузии ионов, в которых обычно используются углеродные электроды, в качестве которых обычно используются углеродные электроды, в качестве которых обычно используются углеродные электроды, в качестве которых обычно используются углеродные электроды, в качестве которых обычно используются углеродные электроды, в качестве которых обычно используются и более и численно реализован метод поиска метода емкостной деионизации водного раствора, как поверхности конденсатора большой емкости, по которому происходит движение раствора, как поверхности конденсатора, которых обычно используются углеродные электроды, в качестве которых обычно используются и линейно-краевые задачи, как поверхности конденсатора, по которому происходит движение раствора, как поверхности конденсатора, по которому происходит движение раствора, как поверхност

Модель, обученная на 1 эпохе (lr=1e-5):

In [ ]:
input = tokenizer_3(txt, add_special_tokens=True, max_length=2890, padding="max_length", truncation=True, return_tensors='pt').to(device)
tokenizer_3.decode(model_t5_3.generate(input_ids=input.input_ids, attention_mask=input.attention_mask, max_length=500)[0], skip_special_tokens=True)

'Выпускная квалификационная работа рассматривается метод емкостной деионизации, которая именно построен алгоритм количественного расчета условий равновесия, переноса и диффузии ионов, в которых обычно используются углеродные электроды, в качестве которых обычно используются углеродные электроды, в качестве которых обычно используются углеродные электроды, в качестве которых обычно используются углеродные электроды, в качестве которых обычно используются углеродные электроды, в качестве которых обычно используются и более и численно реализован метод поиска метода емкостной деионизации водного раствора, как поверхности конденсатора большой емкости, по которому происходит движение раствора, как поверхности конденсатора, которых обычно используются углеродные электроды,'

*Никаких изменений*

Модель, обученная на 2-х эпохах (lr уменьшен до lr=1e-4):

In [ ]:
input = tokenizer_3(txt, add_special_tokens=True, max_length=2890, padding="max_length", truncation=True, return_tensors='pt').to(device)
tokenizer_3.decode(model_t5_3.generate(input_ids=input.input_ids, attention_mask=input.attention_mask, max_length=500)[0], skip_special_tokens=True)

'Метод емкостной деионизации — менее энергоемкий и экономичный метод опреснения воды в сравнении с методами обратного осмоса и дистилляции. Это более энергоемкий и экономичный метод опреснения воды в сравнении с методами обратного осмоса и дистилляции.'

*Есть изменения*

Модель, обученная на 3-х эпохах:

In [ ]:
input = tokenizer_3(txt, add_special_tokens=True, max_length=2890, padding="max_length", truncation=True, return_tensors='pt').to(device)
tokenizer_3.decode(model_t5_3.generate(input_ids=input.input_ids, attention_mask=input.attention_mask, max_length=500)[0], skip_special_tokens=True)

'В работе рассматривается метод емкостной деионизации — менее энергоемкий и экономичный метод опреснения воды в сравнении с методами обратного осмоса и дистилляции. Это более энергоемкий и экономичный метод опреснения воды в сравнении с методами обратного осмоса и дистилляции.'

*Модель начала заикаться...*

Модель, обученная на 4-х эпохах (можно считать, что на 3-х - без первой):

In [ ]:
input = tokenizer_3(txt, add_special_tokens=True, max_length=2890, padding="max_length", truncation=True, return_tensors='pt').to(device)
tokenizer_3.decode(model_t5_3.generate(input_ids=input.input_ids, attention_mask=input.attention_mask, max_length=500)[0], skip_special_tokens=True)

'В работе рассматривается метод емкостной деионизации водного раствора, которые можно будет пропускать водного раствора через электрохимическую ячейку между двумя пористыми электродами. Это более энергоемкий метод опреснения воды в сравнении с методами обратного осмоса и дистилляции.'

*Избавились от заикания, ответ весьма интересный.*

### Пример оценки Rouge:

In [ ]:
txt = 'В работе рассматривается метод емкостной деионизации водного раствора, которые можно будет пропускать водного раствора через электрохимическую ячейку между двумя пористыми электродами. Это более энергоемкий метод опреснения воды в сравнении с методами обратного осмоса и дистилляции.'
targ = 'В работе рассматривается метод емкостной деионизации — менее энергоемкий и экономичный метод опреснения воды в сравнении с методами обратного осмоса и дистилляции. Суть метода емкостной деионизации подробно рассмотрена в работе. Он заключается в пропускании водного раствора через электрохимическую ячейку между двумя пористыми электродами, в качестве которых обычно используются углеродные электроды с высокой удельной поверхностью, между которыми задается определённая разность потенциалов.'

m = Rouge(variants=["L", 2], multiref="best")

candidate = txt.split()
references = [ targ.split(), ]
m.update(([candidate], [references]))

print(m.compute())

{'Rouge-L-P': 0.5142857142857142, 'Rouge-L-R': 0.3, 'Rouge-L-F': 0.3, 'Rouge-2-P': 0.6470588235294118, 'Rouge-2-R': 0.3728813559322034, 'Rouge-2-F': 0.3728813559322034}


## Дообучение на своем датасете:

Функция, написанная вручную, подает на обучение по 1 тексту за раз (обучение на 10 текстах, версия pro):

In [ ]:
def train(model, tokenizer, train_dataset, epochs=2, report_steps=200):

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    model.train()
    losses = 0

    for epoch in range(epochs):
        for batch in tqdm(train_dataset):
            txts, refs = batch
            txts = tokenizer(txts, return_tensors='pt', padding = True).to(device)
            refs = tokenizer(refs, return_tensors='pt', padding = True).to(device)
            refs.input_ids[refs.input_ids == 0] = -100
            loss = model(
                input_ids=txts.input_ids,
                attention_mask=txts.attention_mask,
                labels=refs.input_ids,
                decoder_attention_mask=refs.attention_mask,
                return_dict=True
            ).loss
            loss.backward()
            optimizer.zero_grad()

            torch.cuda.empty_cache()

            losses += loss.item()
            loss.detach()

Запускаем обучение:

In [ ]:
train(model_t5_3, tokenizer_3, train_set, epochs=1)

100%|██████████| 10/10 [00:03<00:00,  2.66it/s]


Текст для тестирования:

In [ ]:
with open(f'{path}0.txt') as f:
    data = f.read()
data = data.split('\n\n')
txt, ref = data[1], data[2]

In [ ]:
ref

In [ ]:
model_t5_3.eval()
input = tokenizer_3(txt, return_tensors='pt', padding = True).to(device)
tokenizer_3.decode(model_t5_3.generate(input_ids=input.input_ids, attention_mask=input.attention_mask)[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Если именно можно объяснить, что именно можно определить,'

# PEGASUS

https://huggingface.co/docs/transformers/model_doc/pegasus

In [ ]:
from transformers import AutoTokenizer, TFPegasusForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

print(tokenizer(txt, return_tensors="tf")["input_ids"].shape)

Token indices sequence length is longer than the specified maximum sequence length for this model (23861 > 512). Running this sequence through the model will result in indexing errors


(1, 23861)


 PegasusForConditionalGeneration



In [ ]:
from transformers import AutoTokenizer, TFPegasusForConditionalGeneration

model = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

ARTICLE_TO_SUMMARIZE = (

    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "

    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "

    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."

)

inputs = tokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="tf")

# Generate Summary

summary_ids = model.generate(inputs["input_ids"])

tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
"California's largest electricity provider has turned off power to hundreds of thousands of customers."

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.9/dist-packages/transformers/generation/tf_utils.py:854: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"California's largest electricity provider has turned off power to hundreds of thousands of customers."

In [ ]:
model.to(device)
clear_output()

Текст на русском не обрабатывает, но забавно:

In [ ]:
ARTICLE_TO_SUMMARIZE = txt

inputs = tokenizer(ARTICLE_TO_SUMMARIZE, padding = "max_length", max_length=1024, truncation=True, return_tensors="tf")

# Generate Summary

summary_ids = model.generate(inputs["input_ids"])

print(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False))

['All photographs courtesy of AFP, EPA, Getty Images and Reuters']


In [ ]:
model.to('cpu')
clear_output()

Ломается на длинных текстах:

In [ ]:
ARTICLE_TO_SUMMARIZE = (

"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."
"Text analytics has become increasingly popular in recent years because of the ubiquity of text data on the Web, social networks, emails, digital libraries, and chat sites. The extraction of useful insights from text with various types of statistical algorithms is referred to as text mining, text analytics, or machine learning from text."
"Numerous applications exist in the context of the types of insights one of trying to discover from a text collection, such as search engines, text mining, recommender systems, opinion mining and sentiment analysis. The area of text mining is closely related to that of information retrieval, although the latter topic focuses on the database management issues rather than the mining issues."
"Most machine learning applications in the text domain work with the bag-of-words representation in which the words are treated as dimensions with values corresponding to word frequencies. A data set corresponds to a collection of documents, which is also referred to as a corpus. The complete and distinct set of words used to define the corpus is also referred to as the lexicon. Dimensions are also referred to as terms or features. Some applications of text work with a binary representation in which the presence of a term in a document corresponds to a value of 1, and 0, otherwise. Other applications use a normalized function of the word frequencies as the values of the dimensions. In each of these cases, the dimensionality of data is very large, and may be of the order of 105 or even 106. Furthermore, most values of the dimensions are 0s, and only a few dimensions take on positive values. In other words, text is a high-dimensional, sparse, and non-negative representation."

)

inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="pt")

# Generate Summary

summary_ids = model.generate(inputs["input_ids"])

print(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False))

IndexError: ignored

Модель ниже много весит, не влезает в обычную версию colab:

## PEGASUS-X

https://huggingface.co/docs/transformers/model_doc/pegasus_x

PegasusXConfig example

In [ ]:
from transformers import PegasusXConfig, PegasusXModel

# Initializing a PEGASUS google/pegasus-x-large style configuration

configuration = PegasusXConfig()

# Initializing a model (with random weights) from the google/pegasus-x-large style configuration

model = PegasusXModel(configuration)

# Accessing the model configuration

configuration = model.config

PegasusXModel example

In [ ]:
from transformers import AutoTokenizer, PegasusModel

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-large")

model = PegasusModel.from_pretrained("google/pegasus-x-large")

inputs = tokenizer("Studies have been shown that owning a dog is good for you", return_tensors="pt")

decoder_inputs = tokenizer("Studies show that", return_tensors="pt")

outputs = model(input_ids=inputs.input_ids, decoder_input_ids=decoder_inputs.input_ids)

last_hidden_states = outputs.last_hidden_state

list(last_hidden_states.shape)
[1, 4, 1024]

 PegasusXForConditionalGeneration (summarization) example

In [ ]:
from transformers import AutoTokenizer, PegasusXForConditionalGeneration

model = PegasusXForConditionalGeneration.from_pretrained("google/pegasus-x-base").to(device)

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-large")

ARTICLE_TO_SUMMARIZE = (

    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "

    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "

    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."

)

inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=64, return_tensors="pt", truncation=True).to(device) #max_new_tokens

print(inputs)

# Generate Summary

summary_ids = model.generate(inputs["input_ids"])

tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
"California's largest electricity provider has turned off power to hundreds of thousands of customers."

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 16384 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
inputs